In [83]:
import sys
import pandas as pd
import numpy as np
import datetime
from sodapy import Socrata
import geopandas
import matplotlib
from shapely.geometry import Point
import matplotlib.pyplot as plt
import requests

In [ ]:
pd.options.display.max_rows = 15
pd.options.display.max_columns = 15

## Part 1

In [5]:
#Retrieve Data from API (chicago data portal)
client = Socrata("data.cityofchicago.org", None)
results = client.get("6zsd-86xi", where="year=2017 OR year=2018", limit=1000000)
results_df = pd.DataFrame.from_records(results)

In [8]:
#clean dates
results_df['date_formatted'] = pd.to_datetime(results_df['date'])
results_df['month'] = results_df['date_formatted'].dt.month
results_df['week'] = results_df['date_formatted'].dt.week

#specify types and order for each column
clean = results_df[['id', 
                  'case_number',
                  'date_formatted',
                  'month',
                  'week',
                  'year',
                  'block', 
                  'iucr',
                  'primary_type',
                  'description',
                  'location_description',
                  'arrest',
                  'domestic',
                  'beat',
                  'district',
                  'ward',
                  'community_area',
                  'fbi_code',
                  'x_coordinate',
                  'y_coordinate',
                  'latitude',
                  'longitude']]

convert_dict = {
                'id' : 'int',
              'case_number': 'str',
              'block': 'str', 
              'iucr' : 'str',
              'primary_type' : 'str',
              'description' : 'str',
              'location_description' :'str',
              'arrest' : 'bool',
              'domestic' : 'bool',
              'beat' : 'str' ,
              'district' : 'str',
              'ward' : 'str',
              'community_area' : 'str',
              'fbi_code' : 'str',
              'x_coordinate' : 'float',
              'y_coordinate' : 'float',
              'latitude' : 'float',
              'longitude' : 'float'}

clean = clean.astype(convert_dict)

In [139]:
# Create Summary statistics of type of crime per year
year_groups = clean.groupby(['year', 'primary_type'])
value_counts = year_groups.agg({'primary_type' : 'count'})
by_type = value_counts.unstack(level=0)
by_type = by_type['primary_type'].reset_index()
by_type["percent_change"] = (by_type['2018'] - by_type['2017'])/by_type['2017']
by_type = by_type.sort_values(by="2017", ascending=False)
by_type

year,primary_type,2017,2018,percent_change
30,THEFT,64346,65082,0.011438
2,BATTERY,49214,49782,0.011541
6,CRIMINAL DAMAGE,29042,27806,-0.042559
1,ASSAULT,19303,20377,0.055639
8,DECEPTIVE PRACTICE,19028,18733,-0.015503
23,OTHER OFFENSE,17227,17130,-0.005631
3,BURGLARY,13001,11730,-0.097762
...,...,...,...,...
20,OBSCENITY,87,86,-0.011494
4,CONCEALED CARRY LICENSE VIOLATION,69,149,1.159420


The top 5 types of crimes are theft, battery, criminal damage, assault, and deceptive practice. Theft, battery, and assault increased slightly between 2017 and 2018, and criminal damage and deceptive practice decreased slightly.

In [55]:
total_crimes = by_type.agg({"2017" : "sum", "2018" : "sum"}).to_frame(name="total_crimes").reset_index()
total_crimes

,index,total_crimes
0,2017,268098
1,2018,266275


# Come back to this and do more descriptive stuff.

# Part 2

In [77]:
#Add block groups to the crimes dataframe

#first create a location point in the crimes dataframe
clean['coordinates'] = list(zip(clean.longitude, clean.latitude))
clean['coordinates'] = clean['coordinates'].apply(Point)
crimes_loc = geopandas.GeoDataFrame(clean, geometry='coordinates')


missing_gps = crimes_loc.latitude.notnull()
crimes_loc = crimes_loc[missing_gps]

In [84]:
#read in shapefile, downloaded from https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2018&layergroup=Block+Groups
block_shape = geopandas.read_file("tl_2018_17_bg/tl_2018_17_bg.shp")

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,17,175,951500,3,171759515003,Block Group 3,G5030,S,267762910,12295,+41.0283710,-089.8495082,"POLYGON ((-89.985479 40.977203, -89.9854879999..."
1,17,175,951400,4,171759514004,Block Group 4,G5030,S,23892876,0,+41.0998489,-089.8409271,"POLYGON ((-89.87840199999999 41.106688, -89.87..."
2,17,175,951400,3,171759514003,Block Group 3,G5030,S,156690063,322516,+41.1420518,-089.8638280,"POLYGON ((-89.98491399999999 41.084687, -89.98..."
3,17,175,951400,1,171759514001,Block Group 1,G5030,S,85399097,0,+41.2067522,-089.7212073,"POLYGON ((-89.810795 41.22781699999999, -89.81..."
4,17,183,011100,1,171830111001,Block Group 1,G5030,S,84897206,320477,+39.9921826,-087.5773378,"POLYGON ((-87.618442 39.999161, -87.6182269999..."
5,17,057,953800,1,170579538001,Block Group 1,G5030,S,14971296,0,+40.4064006,-090.1595371,"POLYGON ((-90.190433 40.39615, -90.19041399999..."
6,17,057,952800,3,170579528003,Block Group 3,G5030,S,2564379,13105,+40.6884889,-089.9958612,"POLYGON ((-90.00611599999999 40.68412, -90.006..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9684,17,031,770602,1,170317706021,Block Group 1,G5030,S,2553005,0,+42.0175137,-087.9098032,"POLYGON ((-87.92084199999999 42.022917, -87.91..."
9685,17,043,841111,2,170438411112,Block Group 2,G5030,S,1192265,2052,+41.9558485,-088.0790904,"POLYGON ((-88.09482299999999 41.955281, -88.09..."


In [89]:
crimes_w_blocks = geopandas.sjoin(crimes_loc, block_shape , how="inner", op='intersects')

In [80]:
crimes_w_blocks

,id,case_number,date_formatted,month,week,year,block,...,UACE10,UATYPE,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10
0,11162428,JA529032,2017-11-28 21:43:00,11,48,2017,026XX S CALIFORNIA BLVD,...,16264,U,S,10654,0,+41.8436252,-087.6949060
5278,10808658,JA107780,2017-01-07 21:15:00,1,1,2017,026XX S CALIFORNIA AVE,...,16264,U,S,10654,0,+41.8436252,-087.6949060
6728,10811515,JA110662,2017-01-10 13:00:00,1,2,2017,026XX S CALIFORNIA AVE,...,16264,U,S,10654,0,+41.8436252,-087.6949060
8392,10823456,JA124570,2017-01-21 10:00:00,1,3,2017,026XX S CALIFORNIA AVE,...,16264,U,S,10654,0,+41.8436252,-087.6949060
9277,10824626,JA126061,2017-01-22 20:15:00,1,3,2017,026XX S CALIFORNIA AVE,...,16264,U,S,10654,0,+41.8436252,-087.6949060
13667,10832208,JA134083,2017-01-29 09:35:00,1,4,2017,026XX S CALIFORNIA AVE,...,16264,U,S,10654,0,+41.8436252,-087.6949060
14622,10833724,JA135717,2017-01-30 16:42:00,1,5,2017,026XX S CALIFORNIA AVE,...,16264,U,S,10654,0,+41.8436252,-087.6949060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531678,11561694,JC110740,2018-12-04 17:30:00,12,49,2018,021XX W 83RD ST,...,16264,U,S,5557,0,+41.7419874,-087.6774219
531882,11562774,JC111833,2018-12-24 10:00:00,12,52,2018,019XX S CANALPORT AVE,...,16264,U,S,6288,0,+41.8570178,-087.6422350


In [91]:
crimes_w_blocks.columns

Index(['id', 'case_number', 'date_formatted', 'month', 'week', 'year', 'block',
       'iucr', 'primary_type', 'description', 'location_description', 'arrest',
       'domestic', 'beat', 'district', 'ward', 'community_area', 'fbi_code',
       'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'coordinates',
       'index_right', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')

### Retrieve ACS Data

In [136]:
request_obj = requests.get("https://api.census.gov/data/2017/acs/acs5?get=NAME,B03002_001E,B03002_003E,B03002_012E,B03002_004E,B23025_003E,B23025_004E,B23025_005E,B19013_001E&for=block%20group:*&in=state:17%20county:031")
json_obj = request_obj.json()
acs_data = pd.DataFrame(json_obj)
#clean up
new_header = acs_data.iloc[0] 
acs_data = acs_data[1:] 
acs_data.columns = new_header
rename_dict = {"B01003_001E" : "tot_pop", "B03002_003E" : "non_his_white", "B03002_001E" : "hispanic", "B03002_004E" : "black",
               "B23025_003E" : "labor_force", "B23025_004E" : "employed", "B23025_005E" : "unemployed", "B19013_001E" : "median_hh_income"}
acs_data = acs_data.rename(columns=rename_dict)

### Merge with crime data

In [124]:
acs_data.columns


Index(['NAME', 'hispanic', 'non_his_white', 'B03002_012E', 'black',
       'employment', 'employed', 'unemployed', 'median_hh_income', 'state',
       'county', 'tract', 'block group'],
      dtype='object', name=0)

In [141]:
crimes_w_blocks.columns


Index(['id', 'case_number', 'date_formatted', 'month', 'week', 'year', 'block',
       'iucr', 'primary_type', 'description', 'location_description', 'arrest',
       'domestic', 'beat', 'district', 'ward', 'community_area', 'fbi_code',
       'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'coordinates',
       'index_right', 'STATEFP', 'COUNTYFP', 'tract', 'block group', 'GEOID',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')

In [133]:
crimes_w_blocks = crimes_w_blocks.rename(columns={'TRACTCE' : "tract", 'BLKGRPCE': 'block group'})
crimes_w_blocks.columns

Index(['id', 'case_number', 'date_formatted', 'month', 'week', 'year', 'block',
       'iucr', 'primary_type', 'description', 'location_description', 'arrest',
       'domestic', 'beat', 'district', 'ward', 'community_area', 'fbi_code',
       'x_coordinate', 'y_coordinate', 'latitude', 'longitude', 'coordinates',
       'index_right', 'STATEFP', 'COUNTYFP', 'tract', 'block group', 'GEOID',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON'],
      dtype='object')

In [137]:
crimes_acs = crimes_w_blocks.merge(acs_data, how='left', on=["tract", "block group"])

In [138]:
crimes_acs

,id,case_number,date_formatted,month,week,year,block,...,black,labor_force,employed,unemployed,median_hh_income,state,county
0,11162428,JA529032,2017-11-28 21:43:00,11,48,2017,026XX S CALIFORNIA BLVD,...,6729,530,490,40,25714,17,031
1,11211608,JB126459,2018-01-23 08:05:00,1,4,2018,027XX S CALIFORNIA AVE,...,6729,530,490,40,25714,17,031
2,10808658,JA107780,2017-01-07 21:15:00,1,1,2017,026XX S CALIFORNIA AVE,...,6729,530,490,40,25714,17,031
3,10811515,JA110662,2017-01-10 13:00:00,1,2,2017,026XX S CALIFORNIA AVE,...,6729,530,490,40,25714,17,031
4,10818154,JA118537,2017-01-15 16:00:00,1,2,2017,031XX W 27TH ST,...,6729,530,490,40,25714,17,031
5,10822435,JA123049,2017-01-20 12:30:00,1,3,2017,027XX S WHIPPLE ST,...,6729,530,490,40,25714,17,031
6,10823388,JA124307,2017-01-21 12:15:00,1,3,2017,028XX S TROY ST,...,6729,530,490,40,25714,17,031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528515,11430249,JB414341,2018-08-29 20:59:00,8,35,2018,034XX W 117TH ST,...,303,300,272,28,28611,17,031
528516,11494343,JB498629,2018-10-31 12:30:00,10,44,2018,034XX W 117TH ST,...,303,300,272,28,28611,17,031


Based on this augmented data, provide some descriptive statistics to describe:
1. What types of blocks have reports of “Battery”?
2. What types of blocks get “Homicide”?
3. Does that change over time in the data you collected?
4. What is the difference in blocks that get “Deceptive Practice” vs “Sex Offense”?

In [ ]:
year_groups = clean.groupby(['year', ''])
value_counts = year_groups.agg({'primary_type' : 'count'})
by_type = value_counts.unstack(level=0)
by_type = by_type['primary_type'].reset_index()
by_type["percent_change"] = (by_type['2018'] - by_type['2017'])/by_type['2017']
by_type = by_type.sort_values(by="2017", ascending=False)